In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('finger-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))


rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]


ix = float(config.get('Configuration', 'ix'))
iy = float(config.get('Configuration', 'iy'))

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]


In [2]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  



In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))


ts=list(range(t0,tN,time_step))

def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

day=list(range(day_time(t0),day_time(tN)+1))[0]


2024-07-03 18:38:30 End date is 1720024710 day: 19907 2024-07-03 18:38:30
2024-07-03 18:02:15
2024-07-03 18:02:15 Init date is 1720022535 day: 19907 2024-07-03 18:02:15


In [4]:
S={}
for user in users:
    S[user]={}
    series=read_csv(os.path.join(data_folder+"/DAY_"+str(day),"GT.ROOM."+user+".tsv"), sep="\t",parse_dates=True, header=None, names=["tt0","time", "scene"]) 
 
    for index, row in series.iterrows():
        S[user][row["time"]]=row["scene"]
#print(S)

In [5]:
def getFPImg(tt):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

def getGTImg(tt,user):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0  

def getSceneImg(scene):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/FP.mean."+str(scene)+".png"))/255.0

In [6]:



rssi0={}
Y0={}

T0={}
T1={}
S0={}



anchors_data={}
for user in users:

    rssi0[user]=[]
    Y0[user]=[]
    S0[user]=[]
    T1[user]=[]
    T0[user]=[]    


    anchors_data[user]={}
    for anchor in anchors:
        print(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
        print(user,anchor)
        series=read_csv(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d","d2"])
        series=series.values
        anchors_data[user][anchor]=series


for it,tt in enumerate(ts):


    #img00=np.zeros((w,h))
    #scene=getScene(tt)
    #if(scene):
    #    img00=getSceneImg(scene)
    #print("img00",img00)
    #imgX=imgX-img00
    #imgX[imgX<0]=0
    for user in users:
        
        label=S[user][tt]

        imgY=getGTImg(tt,user)
        print("user",user,"imgY",imgY.shape, label)
        #if(label is None or label=="IDLE"):
        #    continue

        dist=[]
        for ixa,anchor in enumerate(anchors):
            #print(tt,time2str(tt),anchor,user)
            d=anchors_data[user][anchor][it][1]
            #rint(anchors_data[user][anchor][it][0], "vs",tt)

            #if(d!=-1):
            #    d=(d - rssi_min) / (rssi_max - rssi_min)

            #print(anchors_data[user][anchor][it][0],anchors_data[user][anchor][it][1],anchors_data[user][anchor][it][2])
            if(d>=0):
                if(d<10):
                    d=(1-d/10.0)
                else:
                    d=0
            else:
                    d=0

            #print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
            dist.append(d)


        
        #dist2=[]
        for ixa,anchor in enumerate(anchors):
            #print(tt,time2str(tt),anchor,user)
            d=anchors_data[user][anchor][it][2]
            #rint(anchors_data[user][anchor][it][0], "vs",tt)

            #if(d!=-1):
            #    d=(d - rssi_min) / (rssi_max - rssi_min)

            #print(anchors_data[user][anchor][it][0],anchors_data[user][anchor][it][1],anchors_data[user][anchor][it][2])
            if(d>=0):
                if(d<10):
                    d=(1-d/10.0)
                else:
                    d=0
            else:
                    d=0

            #print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
            dist.append(d)

        dist=np.array(dist)
        #print("dist:",dist)

        rssi0[user].append(dist.reshape(len(anchors*2),1))

        Y0[user].append(imgY[:, :, np.newaxis])
        T0[user].append(it)
        T1[user].append(tt)
        S0[user].append(label)
        


./data//DAY_19907/ed9c.1fa0.tsv
ed9c 1fa0
./data//DAY_19907/ed9c.2eff.tsv
ed9c 2eff
./data//DAY_19907/ed9c.399a.tsv
ed9c 399a
./data//DAY_19907/ed9c.45d2.tsv
ed9c 45d2
./data//DAY_19907/ed9c.6aca.tsv
ed9c 6aca
./data//DAY_19907/ed9c.8171.tsv
ed9c 8171
./data//DAY_19907/ed9c.a304.tsv
ed9c a304
./data//DAY_19907/ed9c.c492.tsv
ed9c c492
./data//DAY_19907/ed9c.eb38.tsv
ed9c eb38
./data//DAY_19907/0001.1fa0.tsv
0001 1fa0
./data//DAY_19907/0001.2eff.tsv
0001 2eff
./data//DAY_19907/0001.399a.tsv
0001 399a
./data//DAY_19907/0001.45d2.tsv
0001 45d2
./data//DAY_19907/0001.6aca.tsv
0001 6aca
./data//DAY_19907/0001.8171.tsv
0001 8171
./data//DAY_19907/0001.a304.tsv
0001 a304
./data//DAY_19907/0001.c492.tsv
0001 c492
./data//DAY_19907/0001.eb38.tsv
0001 eb38
./data//DAY_19907/3d57.1fa0.tsv
3d57 1fa0
./data//DAY_19907/3d57.2eff.tsv
3d57 2eff
./data//DAY_19907/3d57.399a.tsv
3d57 399a
./data//DAY_19907/3d57.45d2.tsv
3d57 45d2
./data//DAY_19907/3d57.6aca.tsv
3d57 6aca
./data//DAY_19907/3d57.8171.tsv
3d

user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.     0.     0.     0.     0.3855 0.7303 0.     0.5242 0.     0.
 0.     0.     0.3851 0.7261 0.     0.4085]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.859  0.2215 0.     0.     0.     0.7265 0.     0.5645 0.8543 0.1845
 0.     0.     0.     0.7237 0.     0.4535]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0.     0.4731 0.     0.     0.3841 0.1353 0.     0.8001 0.     0.3926
 0.     0.     0.3359 0.1353 0.     0.7958]
user 3d57 imgY (18, 22) s2
dist: [0.8562 0.1967 0.     0.     0.     0.     0.     0.5228 0.8562 0.1967
 0.     0.     0.     0.     0.     0.5228]
user ed9c imgY (18, 22) s2
dist: [0

user 3d57 imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.8001 0.0908 0.     0.     0.     0.8314 0.     0.4488 0.8001 0.0908
 0.     0.     0.     0.8314 0.     0.4488]
user ed9c imgY (18, 22) s2
dist: [0.     0.4596 0.     0.1264 0.2529 0.     0.     0.8094 0.     0.4596
 0.     0.1264 0.2529 0.     0.     0.8094]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.8117 0.11   0.     0.     0.     0.8043 0.     0.0519 0.8062 0.1006
 0.     0.     0.     0.7771 0.     0.0336]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.8356 0.1015 0.1072 0.     0.     0.7977 0.     0.0505 0.8188 0.1015
 0.1072 

user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0.     0.2314 0.     0.     0.5359 0.0566 0.     0.5842 0.     0.2314
 0.     0.     0.5359 0.0566 0.     0.5842]
user 3d57 imgY (18, 22) s2
dist: [0.     0.3827 0.     0.     0.3968 0.     0.     0.7836 0.     0.3827
 0.     0.     0.3968 0.     0.     0.7836]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0.     0.2927 0.     0.     0.5392 0.0871 0.     0.646  0.     0.2927
 0.     0.     0.5392 0.0871 0.     0.646 ]
user 3d57 imgY (18, 22) s2
dist: [0 0 0 0 0 0

user 3d57 imgY (18, 22) s2
dist: [0.     0.4076 0.     0.     0.4085 0.0973 0.     0.7495 0.     0.4029
 0.     0.     0.3743 0.0973 0.     0.749 ]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.     0.4206 0.     0.     0.3813 0.     0.     0.7457 0.     0.4146
 0.     0.     0.3705 0.     0.     0.7088]
user ed9c imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s2
dist: [0.     0.4764 0.     0.     0.4048 0.     0.     0.7452 0.     0.4389
 0.     0.     0.3565 0.     0.     0.7008]
user ed9c imgY (18, 22) s2
dist: [0.     0.6404 0.     0.2201 0.     0.     0.7818 0.     0.     0.6404
 0.     0.2201 0.     0.     0.7818 0.    ]
imgX (18, 22)
user f02e imgY (18, 22) s2
dist: [0.     0.4155 0.     0.     0.     0.4192 0.     0.475  0.     0

...

imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.1939 0.     0.8543 0.     0.     0.     0.4643 0.     0.1939 0.
 0.8393 0.     0.     0.     0.4642 0.    ]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.     0.8539 0.     0.     0.     0.4717 0.     0.     0.
 0.8539 0.     0.     0.     0.4558 0.    ]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.     0.8548 0.     0.     0.     0.474  0.     0.     0.
 0.8525 0.     0.     0.     0.4708 0.    ]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.

user ed9c imgY (18, 22) s3
dist: [0.     0.     0.8426 0.     0.     0.     0.6292 0.     0.     0.
 0.8426 0.     0.     0.     0.6292 0.    ]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.6877 0.     0.7902 0.     0.     0.4928 0.0472 0.     0.683
 0.     0.7874 0.     0.     0.4928 0.0472]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.6872 0.     0.7954 0.     0.     0.4806 0.0111 0.     0.6835
 0.     0.7907 0.     0.     0.4806 0.0111]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.6877 0.     0.7958 0.     0.     0.4862 0.     0.     0.6835
 0.     0.7912 0.     0.     0.4834 0.    ]
user ed9c imgY (18, 22) s3
dist: [0 

imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0.8141 0.     0.     0.     0.     0.7377 0.5476 0.     0.8141 0.
 0.     0.     0.     0.7377 0.5476 0.    ]
user 3d57 imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s3
dist: [0.     0.6792 0.     0.2089 0.     0.     0.     0.4549 0.     0.6792
 0.     0.2089 0.     0.     0.     0.4549]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) s3
dist: [0.     0.     0.     0.     0.8548 0.     0.     0.2328 0.     0.
 0.     0.     0

user ed9c imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s3
dist: [0.     0.4572 0.     0.0041 0.356  0.1142 0.     0.7106 0.     0.4427
 0.     0.0041 0.3523 0.1142 0.     0.6835]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) s3
dist: [0.     0.4488 0.     0.     0.3921 0.     0.     0.7368 0.     0.4483
 0.     0.     0.386  0.     0.     0.7129]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0.     0.     0.     0.     0.8534 0.1737 0.     0.2187 0.     0.
 0.     0.     0.8534 0.1737 0.     0.2187]
user ed9c imgY (18, 22) s3
dist: [0.     0.4033 0.     0.     0.3049 0.     0.     0.7027 0.     0.4015
 0.   

imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) IDLE
dist: [0.     0.3832 0.     0.     0.3785 0.0997 0.     0.7752 0.     0.3832
 0.     0.     0.3785 0.0997 0.     0.7752]
imgX (18, 22)
user f02e imgY (18, 22) s3
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user ed9c imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
imgX (18, 22)
user f02e imgY (18, 22) IDLE
dist: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
user 3d57 imgY (18, 22) IDLE
dist: [0.     0.0561 0.     0.     0.8454 0.2478 0.     0.1165 0.     0.0561
 0.     0.     0.8454 0.2478 0.     0.1165]
user ed9c imgY (18, 22) IDL

In [7]:
from collections import Counter

for user in users:

    rssi0[user]=np.array(rssi0[user])
    Y0[user]=np.array(Y0[user])
    S0[user]=np.array(S0[user])
    print(Counter(S0[user]))
    T0[user]=np.array(T0[user])
    T1[user]=np.array(T1[user])
    minT=np.min(T0[user])
    maxT=np.max(T0[user])


    print(minT)
    print(maxT)

Counter({'s1': 2175})
0
2174
Counter({'s1': 2031, 'IDLE': 144})
0
2174
Counter({'s1': 2175})
0
2174


In [8]:

rssiT=[]
YT=[]

window_size=6

T=window_size

TA=math.ceil(window_size/2)
TB=math.floor(window_size/2)


print(TA,TB)

SS=[]
for user in users:
    minT=np.min(T0[user])
    maxT=np.max(T0[user])
    for t0 in range(minT+TA,maxT-TB):        
        print(user,label,t0,T0[user][t0],T1[user][t0])

        
        tt=T1[user][t0]

        label=S0[user][t0]
        
        if(label=="IDLE"):
            continue
        #scene=getScene(tt)
        #print("Scene",scene,tt,t)
        #if not scene:
        #    continue

        #t=T0[user][t0]
        #subW=list(range(t-TA,t+TB))
        #print("subW:",subW, "t:",t,t0)
        #if not (set(subW).intersection(set(T0)) == set(subW)):
        #    print("Not data for sliding window!",t0)
        #    continue

        if(all(map(lambda x: x <= 0, rssi0[user][t0-TA:t0+TB].flatten()))):
            print("Distance -1 for RSSI in sliding window!",t0)
            continue

        #if(all(map(lambda x: x <= 0, rssi0[user][t0:t0+TB].flatten()))):
        #    print("Distance -1 for RSSI in sliding window!",t0)
        #    continue

        if(all(map(lambda x: x == 0, Y0[user][t0].flatten()))):
            print("Distance 0 for Y0 in t!",t0)
            continue


        #if(XY0[user][t0-TA:t0+TB].shape[0]!=T):
        #    print("#",XY0[user][t0-TA:t0+TB].shape)
        #    continue
        if(rssi0[user][t0-TA:t0+TB].shape[0]!=T):
            print("@",rssi0[user][t0-TA:t0+TB].shape)
            continue

        #XYT.append(XY0[user][t0-TA:t0+TB])
        rssiT.append(rssi0[user][t0-TA:t0+TB])
        YT.append(Y0[user][t0])
        #print(T1[t0],T0[t0])
        SS.append(label)
#XYT=np.array(XYT)
rssiT=np.array(rssiT)
YT=np.array(YT)    
SS=np.array(SS)    
#print(XYT.shape)
print(rssiT.shape)
print(YT.shape)
print(SS.shape)


3 3
ed9c s1 3 3 1720022538
ed9c s1 4 4 1720022539
ed9c s1 5 5 1720022540
ed9c s1 6 6 1720022541
ed9c s1 7 7 1720022542
ed9c s1 8 8 1720022543
ed9c s1 9 9 1720022544
ed9c s1 10 10 1720022545
ed9c s1 11 11 1720022546
ed9c s1 12 12 1720022547
ed9c s1 13 13 1720022548
ed9c s1 14 14 1720022549
ed9c s1 15 15 1720022550
ed9c s1 16 16 1720022551
ed9c s1 17 17 1720022552
ed9c s1 18 18 1720022553
ed9c s1 19 19 1720022554
ed9c s1 20 20 1720022555
ed9c s1 21 21 1720022556
ed9c s1 22 22 1720022557
ed9c s1 23 23 1720022558
ed9c s1 24 24 1720022559
ed9c s1 25 25 1720022560
ed9c s1 26 26 1720022561
ed9c s1 27 27 1720022562
ed9c s1 28 28 1720022563
ed9c s1 29 29 1720022564
ed9c s1 30 30 1720022565
ed9c s1 31 31 1720022566
ed9c s1 32 32 1720022567
ed9c s1 33 33 1720022568
ed9c s1 34 34 1720022569
ed9c s1 35 35 1720022570
ed9c s1 36 36 1720022571
ed9c s1 37 37 1720022572
ed9c s1 38 38 1720022573
ed9c s1 39 39 1720022574
ed9c s1 40 40 1720022575
ed9c s1 41 41 1720022576
ed9c s1 42 42 1720022577
ed9c s1 43

In [9]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Softmax,Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from keras.losses import mean_squared_error

from keras import backend as K

from sklearn.model_selection import KFold

from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Lambda, Dot,Reshape,Multiply, Input, Conv1D, Conv2D, ConvLSTM2D,ReLU,UpSampling2D, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, Concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K
import math

from tensorflow.keras.losses import CategoricalCrossentropy

2024-07-15 22:28:34.442717: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 22:28:34.466658: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 22:28:34.851607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
K=len(anchors*2)
T=window_size

print(K)
print(T)
rssi = Input(shape=(T,K,1))
print(rssi)

encoded_rssi = TimeDistributed(Conv1D(64, 3, activation='relu', padding='same',
        input_shape=(T, K, 1)))(rssi)
#encoded_rssi = TimeDistributed(MaxPooling1D(2, padding='same'))(encoded_rssi)
encoded_rssi = TimeDistributed(Conv1D(128, 3, activation='relu', padding='same'))(encoded_rssi)
print(encoded_rssi)

#encoded_rssi = Dropout(0.25)(encoded_rssi)
encoded_rssi = TimeDistributed(Flatten())(encoded_rssi)

encoded_rssi=LSTM(256, return_sequences=True)(encoded_rssi)
#encoded_rssi = Dropout(0.25)(encoded_rssi)
encoded_rssi=LSTM(512, return_sequences=True)(encoded_rssi)
#encoded_rssi = Dropout(0.25)(encoded_rssi)
print("lstm",encoded_rssi)
#encoded_rssi=Reshape((T,1,1,128))(encoded_rssi)

#print("lstm",encoded_rssi)

18
6
KerasTensor(type_spec=TensorSpec(shape=(None, 6, 18, 1), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


2024-07-15 22:28:36.917512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-15 22:28:36.931189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-15 22:28:36.931237: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-15 22:28:36.933424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-15 22:28:36.933463: I tensorflow/compile

KerasTensor(type_spec=TensorSpec(shape=(None, 6, 18, 128), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")
lstm KerasTensor(type_spec=TensorSpec(shape=(None, 6, 512), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:1', description="created by layer 'lstm_1'")


2024-07-15 22:28:37.864753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-15 22:28:37.865728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-15 22:28:37.867064: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
print("lstm",encoded_rssi)


x = layers.Flatten()(encoded_rssi)

# Adding a Dense layer to reshape the output into a form suitable for Conv2D
x = layers.Dense(8 * 8 * 48, activation='relu')(x)
x = layers.Reshape((8, 8, 48))(x)

# Adding a series of Conv2D and UpSampling2D layers
x = layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = layers.UpSampling2D(size=(2, 2))(x)  # Now shape (16, 16, 128)

x = layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = layers.UpSampling2D(size=(2, 2))(x)  # Now shape (32, 32, 64)

# Adding Conv2D layers to match the target size
x = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = layers.Conv2D(1, kernel_size=(3, 3), padding='same', activation='sigmoid')(x)

# Crop the output to the target size (18, 22, 1)
decoded = layers.Cropping2D(cropping=((7, 7), (5, 5)))(x)



# Compile model
autoencoder = Model(rssi, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['mse', 'mae'])
#autoencoder.compile(optimizer=Adam(learning_rate=0.001),
#              loss=CategoricalCrossentropy(),
#              metrics=['accuracy'])

lstm KerasTensor(type_spec=TensorSpec(shape=(None, 6, 512), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:1', description="created by layer 'lstm_1'")


In [12]:
print(autoencoder)
autoencoder.save("autoencoder.model3.h5")
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 18, 1)]        0         
                                                                 
 time_distributed (TimeDistr  (None, 6, 18, 64)        256       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 6, 18, 128)       24704     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 6, 2304)          0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 6, 256)            2622464   
                                                             

In [13]:
np.random.seed(42)

from tensorflow.python.framework import ops
from keras.models import load_model


In [14]:
# Inicializamos un array vacío para almacenar todas las predicciones
# Y_pred_total = np.array([])

# Importamos KFold para validación cruzada
# from sklearn.model_selection import KFold

# Imprimimos los valores únicos de la variable SS (escenas)
# print(np.unique(SS))
# print
# for ixs, sc in enumerate(np.unique(SS)):
#     print("SCENE", sc)
    # Obtenemos los índices de entrenamiento y prueba basados en la escena (y luego los barajamos)
#     train_indices = np.where(SS != sc)[0]
#     test_indices = np.where(SS == sc)[0]

    # Barajamos los índices de entrenamiento
#     np.random.shuffle(train_indices)

    # Imprimimos los índices de entrenamiento y prueba
#     print("TRAIN:", train_indices, " TEST:", test_indices)
    
    # Seleccionamos los datos de entrenamiento y prueba
#     XYT_train = XYT[train_indices]
#     rssiT_train = rssiT[train_indices]
#     YT_train = YT[train_indices]

#     XYT_test = XYT[test_indices]
#     rssiT_test = rssiT[test_indices]
#     YT_test = YT[test_indices]
    
    # Cargamos el modelo autoencoder preentrenado
#     autoencoder = load_model("autoencoder.model0.h5", custom_objects={'contrastive_loss3': contrastive_loss3})
    
    # Entrenamos el autoencoder con los datos de entrenamiento y validamos con los datos de prueba
#     history = autoencoder.fit([XYT_train, rssiT_train], YT_train,
#                     batch_size = 16,
#                     validation_data = ([XYT_test, rssiT_test], YT_test),
#                     epochs = 25)
    
    # Hacemos predicciones con el autoencoder usando los datos de prueba
#     YT_prediction = autoencoder.predict([XYT_test, rssiT_test])
#     import matplotlib.pyplot as plt

    # Número de muestras a mostrar
#     K = 5
#     ks = np.random.randint(0, len(test_indices), K)

    # Creamos una figura con K subplots (una fila, tres columnas)
#     fig, axes = plt.subplots(K, 3, figsize=(20, 20))

    # Iteramos sobre cada muestra seleccionada
#     for k in range(0, K):
#         mpAll = XYT_test[ks[k]][T-1].reshape(N, M)

        # Mostramos todos los jugadores en el último frame
#         axes[k, 0].imshow(mpAll, cmap='viridis', interpolation='nearest')
#         axes[k, 0].set_title('All players (last frame)')
#         axes[k, 0].axis('off')

        # Mostramos la etiqueta (verdadero) del último frame del jugador
#         label = rssiT_test[ks[k]][T-1]
#         mpOne = YT_test[ks[k]].reshape(N, M)

#         axes[k, 1].imshow(mpOne, cmap='viridis', interpolation='nearest')
#         axes[k, 1].set_title('True (last frame) of player')
#         axes[k, 1].axis('off')

        # Mostramos la predicción del autoencoder
#         mpPrediction = YT_prediction[ks[k]].reshape(N, M)

#         axes[k, 2].imshow(mpPrediction, cmap='viridis', interpolation='nearest')
#         axes[k, 2].set_title('Prediction of player ')
#         axes[k, 2].axis('off')

    # Ajustamos el espacio entre subplots
#     plt.tight_layout()

    # Mostramos los plots
#     plt.show()

# Imprimimos la distribución de las etiquetas de entrenamiento y prueba
# print(np.bincount(YT_train))
# print(np.bincount(YT_test))


In [15]:
from sklearn.model_selection import train_test_split


train_indices=np.arange(rssiT.shape[0])
np.random.shuffle(train_indices)
print("TRAIN:", train_indices)


rssiT_train=rssiT[train_indices]
YT_train=YT[train_indices]

print(rssiT_train.shape)
print(YT_train.shape)

    
autoencoder = load_model("autoencoder.model3.h5")


history = autoencoder.fit(rssiT_train, YT_train,
                    batch_size=16,
            epochs=200)

autoencoder.save("autoencoder.model3.all.50.h5")

TRAIN: [4742 4824  472 ... 5226 5390  860]
(6357, 6, 18, 1)
(6357, 18, 22, 1)


2024-07-15 22:28:38.354891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-15 22:28:38.355747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-15 22:28:38.356493: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/200


2024-07-15 22:28:39.026451: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-15 22:28:39.027522: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-15 22:28:39.028245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

398/398 [==============================] - 9s 13ms/step - loss: 0.2643 - mse: 0.0676 - mae: 0.1656
Epoch 2/200
398/398 [==============================] - 4s 11ms/step - loss: 0.1019 - mse: 0.0093 - mae: 0.0412
Epoch 3/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0880 - mse: 0.0082 - mae: 0.0360
Epoch 4/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0676 - mse: 0.0061 - mae: 0.0266
Epoch 5/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0615 - mse: 0.0053 - mae: 0.0230
Epoch 6/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0589 - mse: 0.0049 - mae: 0.0214
Epoch 7/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0575 - mse: 0.0047 - mae: 0.0204
Epoch 8/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0565 - mse: 0.0045 - mae: 0.0197
Epoch 9/200
398/398 [==============================] - 4s 11ms/step - loss: 0.0557 - mse: 0.0044 - mae: 0.0192
Epoch 10/200


In [16]:
history = autoencoder.fit(rssiT_train, YT_train,
                    batch_size=16,
            epochs=5)

autoencoder.save("autoencoder.model3.all.50.h5")

Epoch 1/5
398/398 [==============================] - 4s 11ms/step - loss: 0.0418 - mse: 0.0017 - mae: 0.0096
Epoch 2/5
398/398 [==============================] - 4s 11ms/step - loss: 0.0419 - mse: 0.0017 - mae: 0.0096
Epoch 3/5
398/398 [==============================] - 4s 11ms/step - loss: 0.0417 - mse: 0.0017 - mae: 0.0095
Epoch 4/5
398/398 [==============================] - 4s 11ms/step - loss: 0.0418 - mse: 0.0017 - mae: 0.0095
Epoch 5/5
398/398 [==============================] - 4s 11ms/step - loss: 0.0417 - mse: 0.0017 - mae: 0.0095
